<a href="https://colab.research.google.com/github/manishiitg/ML_Experiments/blob/master/recruit/fasttext_text_classify_cv_recruit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Training fastText for classification.

This is just to establish a baseline data source for classification

In [0]:
!pip install Flask
!pip install flask_pymongo

In [0]:
!rm -rf fastText
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && sudo pip install . && sudo python setup.py install

Cloning into 'fastText'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 3585 (delta 23), reused 18 (delta 4), pack-reused 3531
Receiving objects: 100% (3585/3585), 8.10 MiB | 10.01 MiB/s, done.
Resolving deltas: 100% (2248/2248), done.
Processing /content/fastText
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2829938 sha256=9570f81e1dcd4794eb478939be7119a9cb39a14fbba204351cde73ef4fb8fa31
  Stored in directory: /tmp/pip-ephem-wheel-cache-mt_9v47i/wheels/a1/9f/52/696ce6c5c46325e840c76614ee5051458c0df10306987e7443
Successfully built fasttext
running install
running bdist_egg
running egg_info
creating python/fasttext_module/fasttext.egg-info
writing python/fasttext_module/fasttext.egg-info/PKG-INFO
writing dependency_links to python/fasttext_module/fasttext.egg-info/dependency_links.txt
writing requirements to python/fasttext_module/fasttext.egg-in

In [0]:
#!wget wget https://dl.fbaipublicfiles.com/fasttext/data/cooking.stackexchange.tar.gz
#!tar xvzf cooking.stackexchange.tar.gz
# !head cooking.stackexchange.txt

# !rm -rf data
# !mkdir data
# !wget -c http://mattmahoney.net/dc/enwik9.zip -P data
# !unzip data/enwik9.zip -d data     
# !head -c 200 data/fil9 

In [0]:
from flask import Flask
from flask_pymongo import PyMongo
from bs4 import BeautifulSoup
import re
import random

In [0]:
#__label__sauce __label__cheese How much does potato starch affect a cheese sauce recipe?
# __label__food-safety __label__acidity Dangerous pathogens capable of growing in acidic environments
# __label__cast-iron __label__stove How do I cover up the white spots on my cast iron stove?
# __label__restaurant Michelin Three Star Restaurant; but if the chef is not there
# sample format needed


app = Flask(__name__)
app.config["MONGO_URI"] = ""
mongo = PyMongo(app)


def cleanMe(html):
    # create a new bs4 object from the html data loaded
    
    # print(html)
    soup = BeautifulSoup(html, "html.parser")
    # remove all javascript and stylesheet code
    for script in soup(["script", "style"]):
        script.extract()
    # get text
    text = soup.get_text(separator=' ')

    # break into lines and remove leading and trailing space on each
    # lines = (line.strip() for line in text.splitlines())
    lines = text.splitlines()
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    # text = '\n'.join(chunk for chunk in chunks if chunk)
    chunks = filter(None, chunks)
    return " ".join(chunks)

ret = mongo.db.naukri_candidates.find({"extra_data.functional_area": {"$exists": True}})
rows = []
for row in ret:
  rows.append(row)


random.shuffle(rows)

print("total candidates" , len(rows))

open('fastText_cv.train', 'w').close()
open('fastText_functional_area.text', 'w').close()

no_of_data_for_cv_classify = 10000
cv_classify_lines_count = 0

with open("fastText_cv.train", 'a') as the_file:
  with open("fastText_functional_area.text", 'a') as the_file2:
    for row in rows:
      education = row["education"]
      workExperiance = row["workExperiance"]
      summary = row['summary']
      key_skills = row["key_skills"]
      projects = row["projects"]
      if "extra_data" not in row:
        continue

      extra_data = row["extra_data"]
      functional_area = re.sub('[^A-Za-z0-9]+', '-', extra_data["functional_area"])

      summary_str = summary.splitlines()

      education_string = []
      for edu in education:
          # education_string.append(edu["degree"] + " " + edu["specific"] + " " + edu["university"] + " " + edu["year"])
          data = [edu["degree"],edu["specific"],edu["university"],edu["year"]]
          random.shuffle(data)
          education_string.append(" ".join(data))

      work_exp_string = []
      for work in workExperiance:
          # work_exp_string.append(work["company_name"]  + " " + work["designation"] + " " + work["date"] + " " + cleanMe(work["desc"]))
          data = [work["company_name"],work["designation"],work["date"],cleanMe(work["desc"]) ]
          random.shuffle(data) 
          work_exp_string.append(" ".join(data))

      key_skills_str =  " ".join(key_skills.split(",")).replace("Key Skills:","")

      project_str = []
      for project in projects:
          project_str.append(cleanMe(project))

      all_lines = {}

      all_lines["summary"] = " ".join(summary_str)
      all_lines["education"] = " ".join(education_string)
      all_lines["work-exp"] = " ".join(work_exp_string)
      all_lines["key-skills"] = key_skills_str
      all_lines["project"] = " ".join(project_str)

      for label in all_lines.keys():
        if len(all_lines[label]) > 0:
          if label == "work-exp" or label == "summary":
            the_file2.write("__label__"  + functional_area + " " + all_lines[label] + "\n")

          if (cv_classify_lines_count < no_of_data_for_cv_classify):
            the_file.write("__label__" + label + " "  + all_lines[label] + "\n")
          # fasttext alredy does tokenization internally

      cv_classify_lines_count += 1

!tail -n 1000 fastText_cv.train | head -n 20 

total candidates 20817
__label__key-skills  Service Delivery Manager Program Manager IT Operations Project Management Program Management Quality Management Risk Management Requirement Gathering Business Analysis Client Relationship Management Data Center Management Effort Estimation
__label__summary A performance-driven marketing professional with reputation to launch, grow & create mega brands with profitable & sustainable market leadership and implementation of innovative path-breaking initiatives. Possess experience in marketing brands at various levels of product life cycle such as launches, mature brands, brands on decline, premium priced brands & technologically high end brands. Key skills include Marketing Operations,Brand Management,Public Relation,Internal Communication ,Media Relations,Media monitoring,Advertisement Management,hospital management,Revenue Generation,Market Analysis 
__label__education Utkal University B.Com 1996  (Commerce) ICWA (CMA) 2001  (ICWA (CMA)) The Re

In [0]:
ret = mongo.db.indeed_candidates.find({}).limit(1000)

open('fastText_cv.dev', 'w').close()
with open("fastText_cv.dev", 'a') as the_file:
  for row in ret:
    education = row["education"]
    workExperiance = row["workExperience"]
    summary = row['summary']
    skills = []
    if "skills_full_data" in row:
      skills_full_data = row["skills_full_data"]
      for skills_f in skills_full_data:
        skills.extend(skills_f.split(" "))
    else:
      for skill in row["skills"]:
        skills.append(skill['skill'])

    additional = row["additionalInformation"]

    summary_str = summary.splitlines()
    additional_str = additional.splitlines()

    summary_str.extend(additional_str)

    education_string = []
    for edu in education:
        data = []
        for key in edu.keys():
          if edu[key] is not None and (len(edu[key])) != 0:
            data.append(edu[key])

        random.shuffle(data)
        education_string.append(" ".join(data))

    work_exp_string = []
    for work in workExperiance:
      data = []
      for key in work.keys():
        if (len(work[key])) != 0:
          data.append(work[key])
          
      random.shuffle(data) 
      work_exp_string.append(" ".join(data))

    key_skills_str =  " ".join(skills)

    all_lines = {}

    all_lines["summary"] = " ".join(summary_str)
    all_lines["education"] = " ".join(education_string)
    all_lines["work-exp"] = " ".join(work_exp_string)
    all_lines["key-skills"] = key_skills_str

    for label in all_lines.keys():
      if len(all_lines[label]) > 0:
        the_file.write("__label__" + label + " "  + " ".join(all_lines[label].splitlines()) + "\n")

!tail -n 1000 fastText_cv.dev | head -n 50 

__label__work-exp Currently I am working as a Software Engineer in the job portal of HT Media Ltd(Shine.com). I am in the recruiter team of shine. My profile is of a Backend developer. Most of my work is of python. Developing new features, enhancing existing features and optimizing the code as much as possible are some of my responsibilities.  Following are some of the work I have done:-  1) I have developed a side project of shine, Leads Management System(LMS).  2) Developed a system in which the job posted by recruiter would automatically predict whether the job posted is actual or fake. Used various machine learning algorithms and nltk libraries to develop this.  3) I also have the knowledge of scraping websites both statically and dynamically.  4) I have set up project's testing environment on docker  5) I have also set up Jenkins on a server which would work on pull request from GitHub. When the Jenkins job runs, the test environment on docker would run the unit test cases, thus g

In [0]:
!wc fastText_cv.train

   38587  4797079 35765503 fastText_cv.train


In [0]:
import fasttext

def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))


model_cv_classify = fasttext.train_supervised(input="fastText_cv.train", epoch=25, lr=1.0, wordNgrams=2, verbose=2, minCount=1)
print_results(*model_cv_classify.test('fastText_cv.dev'))


N	3650
P@1	0.712
R@1	0.712


In [0]:
model_cv_classify.get_labels()

['__label__key-skills',
 '__label__work-exp',
 '__label__education',
 '__label__summary',
 '__label__project']

In [0]:
model_cv_classify = fasttext.train_supervised(input="fastText_cv.train", lr=1.0, epoch=25, wordNgrams=2, bucket=200000, dim=50, loss='hs')
print_results(*model_cv_classify.test('fastText_cv.dev'))

N	3650
P@1	0.659
R@1	0.659


In [0]:
model_cv_classify = fasttext.train_supervised(input="fastText_cv.train", autotuneValidationFile='fastText_cv.dev', verbose=2)
print_results(*model_cv_classify.test('fastText_cv.dev'))

N	3650
P@1	0.779
R@1	0.779


In [0]:
model.quantize(input="fastText_cv.train", qnorm=True, retrain=True, cutoff=100000)
print_results(*model.test('fastText_cv.dev'))

#https://developers.google.com/machine-learning/crash-course/classification/precision-and-recall
# https://towardsdatascience.com/precision-vs-recall-386cf9f89488

In [0]:
model.predict("working with excellence technosoft pvt limited", k =5)

(('__label__summary', '__label__education'), array([0.99876726, 0.00127274]))

In [0]:
model.predict("b.tech from IIT in 2015", k =5)

(('__label__summary', '__label__education'),
 array([9.99548614e-01, 4.91372019e-04]))

In [0]:
model.predict("software developer", k =5)

(('__label__education', '__label__key-skills'),
 array([9.99635994e-01, 3.93978466e-04]))

In [0]:
model.predict("php html css javascript nodejs", k =5)

(('__label__key-skills', '__label__education'),
 array([0.99859434, 0.00143563]))

In [0]:
!wc fastText_functional_area.text

   37809  8674662 64712992 fastText_functional_area.text


In [0]:
!head -n 27809 fastText_functional_area.text > fastText_functional_area.train
!tail -n 10000 fastText_functional_area.text > fastText_functional_area.dev

In [0]:
!head -n 20 fastText_functional_area.train

In [0]:
!head -n 20 fastText_functional_area.dev

__label__ITES-BPO-Operations-Customer-Service-Telecalling Ensure delivery of KPIs/Sales targets including day to day service levels, customer experience, quality measures and compliance measures Keep Communication and Engagement model in place to ensure all teams understand the performance of the business and also that they understand the needs of their teams Coordinate all vendor management tasks inclusive of working with external vendors and internal employees. Perform as functional bridge amongst external vendors as well as internal stakeholders Ensure enterprise-wide needs for risk assessment and continuing vendor monitoring are being accomplished by see, touch and feel method every month. Also track and drive 100% completion of business essential training (mandatory modules) Ensure to track, measure, report and evaluate vendor performance on daily, weekly and monthly basis. Ensure daily, weekly and monthly stand-up are utilized to monitor the KPI's, process improvement and glide p

In [0]:
model_job_type_classify = fasttext.train_supervised(input="fastText_functional_area.train")
print_results(*model_job_type_classify.test('fastText_functional_area.dev'))

N	10000
P@1	0.574
R@1	0.574


In [0]:
model_job_type_classify = fasttext.train_supervised(input="fastText_functional_area.train", autotuneValidationFile="fastText_functional_area.dev")
print_results(*model_job_type_classify.test('fastText_functional_area.dev'))

N	10000
P@1	0.698
R@1	0.698
